# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

<ipython-input-1-1499ab5789c7>:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter
import timm
#from torchvision.models import wide_resnet50_2, resnet18
#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
sys.path.append('/home/hiroki/research/dcase2021_task2/src/functions')
import common as com
import pytorch_modeler as modeler
from pytorch_model import EfficientNet_b1 as Model
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

In [4]:
#model = timm.create_model('efficientnet_b1', pretrained=True)
#3, 320, 320
#name: (channel_multiplier, depth_multiplier, resolution, dropout_rate)
#(1.4, 1.8, 380, 0.4)

In [5]:
#model.blocks[0][0]

## load config and set logger

In [6]:
log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

## Setting

In [7]:
# Setting seed
modeler.set_seed(42)

In [8]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_dir = INPUT_ROOT + "/dev_data"
add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = os.listdir(dev_dir)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/HDD1TB/research/DCASE_experiments/SSL-Efficientnet_frame257/dcase2021_task2/config.yaml'

In [9]:
torch.cuda.is_available()

True

## make path list

- __train_paths__ ([dict]): Each Machine Type
    - __ToyCar__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) :paths
    - __ToyTrain__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) : paths
    - ...

In [10]:
dev_paths = {}

for machine_type in machine_types:
    # dev train
    dev_train_paths = [f"{dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/train")]
    dev_train_paths = sorted(dev_train_paths)
    
    print(f'Machine {machine_type}', end=',')
    print(f'dev_train_size : {len(dev_train_paths)}', end=', ')
    dev_train_source_paths = [path for path in dev_train_paths if 'source' in path]
    print(f'source_size : {len(dev_train_source_paths)}', end=', ')
    dev_train_target_paths = [path for path in dev_train_paths if 'target' in path]
    print(f'target_size : {len(dev_train_target_paths)}', end=', ')
    # source mode
    dev_train_paths = dev_train_source_paths
    
    # add_dev train
    add_dev_paths = [f"{add_dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{add_dev_dir}/{machine_type}/train")]
    add_dev_paths = sorted(add_dev_paths)
    print(f'adddev_train_size : {len(dev_train_paths)}', end=', ')
    add_dev_source_paths = [path for path in add_dev_paths if 'source' in path]
    print(f'source_size : {len(add_dev_source_paths)}', end=', ')
    add_dev_target_paths = [path for path in add_dev_paths if 'target' in path]
    print(f'target_size : {len(add_dev_target_paths)}')
    # source mode
    add_dev_paths = add_dev_source_paths
    print('==============================================================')
    
    # dev_source valid
    dev_source_paths = [f"{dev_dir}/{machine_type}/source_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/source_test")]
    dev_source_paths = sorted(dev_source_paths)
    
    # dev_target valid
    dev_target_paths = [f"{dev_dir}/{machine_type}/target_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/target_test")]
    dev_target_paths = sorted(dev_target_paths)
    
    # bundle
    dev_paths[machine_type] = {}
    dev_paths[machine_type]['train'] = dev_train_paths + add_dev_paths
    dev_paths[machine_type]['valid_source'] = dev_source_paths
    dev_paths[machine_type]['valid_target'] = dev_target_paths

Machine fan,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3000, source_size : 3000, target_size : 9
Machine gearbox,dev_train_size : 3026, source_size : 3017, target_size : 9, adddev_train_size : 3017, source_size : 3105, target_size : 9
Machine pump,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3000, source_size : 3000, target_size : 9
Machine slider,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3000, source_size : 3000, target_size : 9
Machine ToyCar,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3000, source_size : 3000, target_size : 9
Machine ToyTrain,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3000, source_size : 3000, target_size : 9
Machine valve,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3000, source_size : 3000, target_size : 9


## training

In [11]:
#############################################################################
# run
#############################################################################
def run(machine_type, dev_paths):
    com.tic()
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(dev_paths, machine_type)
    # parameter setting
    logger.info('FEATURE EXTRACTION')
    net = Model(n_out=config['param']['mel_bins'])
    optimizer = optim.Adam(net.parameters(), lr=config['param']['lr'])
    output_dicts = modeler.train_net(net, dataloaders_dict, writer, optimizer)
    
    out_path = f'{MODEL_DIR}/{machine_type}_model.pkl'
    torch.save(output_dicts['net'], out_path)
    logger.info(f'SAVE SUCCESS : {out_path}')
    
    com.toc()

In [12]:
machine_types

['fan', 'gearbox', 'pump', 'slider', 'ToyCar', 'ToyTrain', 'valve']

# run

In [13]:
for machine_type in machine_types:
    run(machine_type, dev_paths)

2021-08-10 23:09:38,980 - 00_train.py - INFO - TARGET MACHINE_TYPE: fan
2021-08-10 23:09:38,981 - 00_train.py - INFO - MAKE DATA_LOADER
2021-08-10 23:09:38,982 - 00_train.py - INFO - FEATURE EXTRACTION


use: cuda:0


100%|██████████| 24/24 [00:24<00:00,  1.02s/it]
2021-08-10 23:10:06,810 - pytorch_modeler.py - INFO - epoch:1/200, tr_loss:501.616866
100%|██████████| 24/24 [00:23<00:00,  1.00it/s]
2021-08-10 23:10:30,805 - pytorch_modeler.py - INFO - epoch:2/200, tr_loss:139.078657
100%|██████████| 24/24 [00:23<00:00,  1.00it/s]
2021-08-10 23:10:54,711 - pytorch_modeler.py - INFO - epoch:3/200, tr_loss:111.630910
100%|██████████| 24/24 [00:24<00:00,  1.02s/it]
2021-08-10 23:11:19,129 - pytorch_modeler.py - INFO - epoch:4/200, tr_loss:95.174875
100%|██████████| 24/24 [00:23<00:00,  1.00it/s]
2021-08-10 23:11:43,038 - pytorch_modeler.py - INFO - epoch:5/200, tr_loss:79.800924
100%|██████████| 24/24 [00:23<00:00,  1.01it/s]
2021-08-10 23:12:06,918 - pytorch_modeler.py - INFO - epoch:6/200, tr_loss:70.560312
100%|██████████| 24/24 [00:23<00:00,  1.01it/s]
2021-08-10 23:12:30,705 - pytorch_modeler.py - INFO - epoch:7/200, tr_loss:66.712590
100%|██████████| 24/24 [00:23<00:00,  1.02it/s]
2021-08-10 23:12:5

100%|██████████| 24/24 [00:19<00:00,  1.25it/s]
2021-08-10 23:30:42,825 - pytorch_modeler.py - INFO - epoch:62/200, tr_loss:27.711652
100%|██████████| 24/24 [00:19<00:00,  1.25it/s]
2021-08-10 23:31:01,995 - pytorch_modeler.py - INFO - epoch:63/200, tr_loss:27.538941
100%|██████████| 24/24 [00:19<00:00,  1.21it/s]
2021-08-10 23:31:21,903 - pytorch_modeler.py - INFO - epoch:64/200, tr_loss:27.521156
100%|██████████| 24/24 [00:19<00:00,  1.25it/s]
2021-08-10 23:31:41,111 - pytorch_modeler.py - INFO - epoch:65/200, tr_loss:28.770995
100%|██████████| 24/24 [00:19<00:00,  1.23it/s]
2021-08-10 23:32:00,570 - pytorch_modeler.py - INFO - epoch:66/200, tr_loss:27.558602
100%|██████████| 24/24 [00:19<00:00,  1.25it/s]
2021-08-10 23:32:19,781 - pytorch_modeler.py - INFO - epoch:67/200, tr_loss:27.045907
100%|██████████| 24/24 [00:19<00:00,  1.25it/s]
2021-08-10 23:32:38,973 - pytorch_modeler.py - INFO - epoch:68/200, tr_loss:26.735026
100%|██████████| 24/24 [00:19<00:00,  1.25it/s]
2021-08-10 23:

100%|██████████| 24/24 [00:19<00:00,  1.25it/s]
2021-08-10 23:50:20,813 - pytorch_modeler.py - INFO - epoch:123/200, tr_loss:24.472764
100%|██████████| 24/24 [00:19<00:00,  1.25it/s]
2021-08-10 23:50:39,999 - pytorch_modeler.py - INFO - epoch:124/200, tr_loss:23.901289
100%|██████████| 24/24 [00:19<00:00,  1.25it/s]
2021-08-10 23:50:59,239 - pytorch_modeler.py - INFO - epoch:125/200, tr_loss:22.714871
100%|██████████| 24/24 [00:19<00:00,  1.20it/s]
2021-08-10 23:51:19,172 - pytorch_modeler.py - INFO - epoch:126/200, tr_loss:24.318049
100%|██████████| 24/24 [00:19<00:00,  1.26it/s]
2021-08-10 23:51:38,278 - pytorch_modeler.py - INFO - epoch:127/200, tr_loss:22.122444
100%|██████████| 24/24 [00:19<00:00,  1.25it/s]
2021-08-10 23:51:57,507 - pytorch_modeler.py - INFO - epoch:128/200, tr_loss:23.130897
100%|██████████| 24/24 [00:19<00:00,  1.24it/s]
2021-08-10 23:52:16,791 - pytorch_modeler.py - INFO - epoch:129/200, tr_loss:21.972109
100%|██████████| 24/24 [00:19<00:00,  1.26it/s]
2021-08

...

100%|██████████| 24/24 [01:23<00:00,  3.47s/it]
2021-08-11 05:43:16,522 - pytorch_modeler.py - INFO - epoch:1/200, tr_loss:592.054875
100%|██████████| 24/24 [00:19<00:00,  1.25it/s]
2021-08-11 05:43:35,759 - pytorch_modeler.py - INFO - epoch:2/200, tr_loss:175.241666
100%|██████████| 24/24 [00:19<00:00,  1.25it/s]
2021-08-11 05:43:54,999 - pytorch_modeler.py - INFO - epoch:3/200, tr_loss:135.207672
100%|██████████| 24/24 [00:19<00:00,  1.26it/s]
2021-08-11 05:44:14,035 - pytorch_modeler.py - INFO - epoch:4/200, tr_loss:111.303912
100%|██████████| 24/24 [00:19<00:00,  1.26it/s]
2021-08-11 05:44:33,116 - pytorch_modeler.py - INFO - epoch:5/200, tr_loss:96.094628
100%|██████████| 24/24 [00:19<00:00,  1.26it/s]
2021-08-11 05:44:52,184 - pytorch_modeler.py - INFO - epoch:6/200, tr_loss:86.418608
100%|██████████| 24/24 [00:19<00:00,  1.20it/s]
2021-08-11 05:45:12,127 - pytorch_modeler.py - INFO - epoch:7/200, tr_loss:79.850785
100%|██████████| 24/24 [00:19<00:00,  1.25it/s]
2021-08-11 05:45:

100%|██████████| 24/24 [00:19<00:00,  1.26it/s]
2021-08-11 06:02:51,852 - pytorch_modeler.py - INFO - epoch:62/200, tr_loss:37.607061
100%|██████████| 24/24 [00:19<00:00,  1.21it/s]
2021-08-11 06:03:11,716 - pytorch_modeler.py - INFO - epoch:63/200, tr_loss:40.924636
100%|██████████| 24/24 [00:19<00:00,  1.25it/s]
2021-08-11 06:03:30,892 - pytorch_modeler.py - INFO - epoch:64/200, tr_loss:39.944113
100%|██████████| 24/24 [00:19<00:00,  1.25it/s]
2021-08-11 06:03:50,076 - pytorch_modeler.py - INFO - epoch:65/200, tr_loss:39.618082
100%|██████████| 24/24 [00:19<00:00,  1.26it/s]
2021-08-11 06:04:09,175 - pytorch_modeler.py - INFO - epoch:66/200, tr_loss:39.683535
100%|██████████| 24/24 [00:19<00:00,  1.26it/s]
2021-08-11 06:04:28,219 - pytorch_modeler.py - INFO - epoch:67/200, tr_loss:37.770354
100%|██████████| 24/24 [00:19<00:00,  1.26it/s]
2021-08-11 06:04:47,236 - pytorch_modeler.py - INFO - epoch:68/200, tr_loss:35.958024
100%|██████████| 24/24 [00:19<00:00,  1.25it/s]
2021-08-11 06:

100%|██████████| 24/24 [00:19<00:00,  1.25it/s]
2021-08-11 06:22:28,531 - pytorch_modeler.py - INFO - epoch:123/200, tr_loss:30.485157
100%|██████████| 24/24 [00:19<00:00,  1.26it/s]
2021-08-11 06:22:47,551 - pytorch_modeler.py - INFO - epoch:124/200, tr_loss:30.131169
100%|██████████| 24/24 [00:19<00:00,  1.24it/s]
2021-08-11 06:23:06,854 - pytorch_modeler.py - INFO - epoch:125/200, tr_loss:29.216557
100%|██████████| 24/24 [00:19<00:00,  1.22it/s]
2021-08-11 06:23:26,486 - pytorch_modeler.py - INFO - epoch:126/200, tr_loss:30.346405
100%|██████████| 24/24 [00:19<00:00,  1.25it/s]
2021-08-11 06:23:45,658 - pytorch_modeler.py - INFO - epoch:127/200, tr_loss:29.909074
100%|██████████| 24/24 [00:19<00:00,  1.24it/s]
2021-08-11 06:24:04,971 - pytorch_modeler.py - INFO - epoch:128/200, tr_loss:30.743293
100%|██████████| 24/24 [00:19<00:00,  1.24it/s]
2021-08-11 06:24:24,275 - pytorch_modeler.py - INFO - epoch:129/200, tr_loss:29.452730
100%|██████████| 24/24 [00:19<00:00,  1.26it/s]
2021-08

100%|██████████| 24/24 [00:19<00:00,  1.22it/s]
2021-08-11 06:42:04,872 - pytorch_modeler.py - INFO - epoch:184/200, tr_loss:24.949050
100%|██████████| 24/24 [00:19<00:00,  1.26it/s]
2021-08-11 06:42:23,893 - pytorch_modeler.py - INFO - epoch:185/200, tr_loss:23.501542
100%|██████████| 24/24 [00:19<00:00,  1.26it/s]
2021-08-11 06:42:43,013 - pytorch_modeler.py - INFO - epoch:186/200, tr_loss:23.415306
100%|██████████| 24/24 [00:19<00:00,  1.26it/s]
2021-08-11 06:43:02,070 - pytorch_modeler.py - INFO - epoch:187/200, tr_loss:23.519431
100%|██████████| 24/24 [00:19<00:00,  1.21it/s]
2021-08-11 06:43:21,954 - pytorch_modeler.py - INFO - epoch:188/200, tr_loss:23.576262
100%|██████████| 24/24 [00:19<00:00,  1.25it/s]
2021-08-11 06:43:41,185 - pytorch_modeler.py - INFO - epoch:189/200, tr_loss:22.964248
100%|██████████| 24/24 [00:19<00:00,  1.25it/s]
2021-08-11 06:44:00,465 - pytorch_modeler.py - INFO - epoch:190/200, tr_loss:23.331367
100%|██████████| 24/24 [00:19<00:00,  1.26it/s]
2021-08

elapsed time: 3920.299005508 [sec]


In [14]:
#run(machine_types[1], dev_paths)